In [1]:
import torch
from torch import nn

In [3]:
net = nn.Sequential(
    nn.Linear(4, 8), 
    nn.ReLU(), 
    nn.Linear(8, 1)
)
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1574],
        [0.1637]], grad_fn=<AddmmBackward0>)

In [12]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.2945, -0.1818, -0.1734,  0.3400, -0.0146, -0.1670,  0.3135, -0.2096]])),
             ('bias', tensor([0.0537]))])

In [22]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0537], requires_grad=True)
tensor([0.0537])


In [25]:
net[2].weight.grad == None

True

In [29]:
print([(name, param.shape) for name, param in net[0].named_parameters()])
print([(name, param.shape) for name, param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


In [42]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.2338,  0.4909, -0.0743,  0.3814],
                      [ 0.1680,  0.0033, -0.4403,  0.2833],
                      [-0.2975, -0.4836, -0.2465,  0.4675],
                      [ 0.2727,  0.0142,  0.0264,  0.1517],
                      [ 0.4470,  0.3017,  0.3078,  0.3225],
                      [-0.0694, -0.3856, -0.4554, -0.0638],
                      [-0.3063, -0.3803,  0.3214, -0.4629],
                      [-0.4488,  0.1561, -0.4244, -0.1974]])),
             ('0.bias',
              tensor([-0.0021, -0.0593, -0.2973,  0.4279,  0.3693,  0.2031, -0.2926,  0.2847])),
             ('2.weight',
              tensor([[-0.2945, -0.1818, -0.1734,  0.3400, -0.0146, -0.1670,  0.3135, -0.2096]])),
             ('2.bias', tensor([0.0537]))])

In [37]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4262],
        [-0.4262]], grad_fn=<AddmmBackward0>)

In [39]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [44]:
rgnet[0][1][0].bias.data
# rgnet.state_dict()

tensor([-0.2149, -0.1922, -0.0722, -0.1649, -0.0202, -0.4847, -0.3163,  0.1047])

In [45]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0033,  0.0080,  0.0002, -0.0017]), tensor(0.))

In [50]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data, net[0].bias.data

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [51]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
net[0].weight.data[0], net[2].weight.data

(tensor([-0.6952, -0.7066,  0.2340, -0.3617]),
 tensor([[42., 42., 42., 42., 42., 42., 42., 42.]]))

In [57]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, -8.0256, -0.0000],
        [ 9.8454,  0.0000,  6.4342,  0.0000]], grad_fn=<SliceBackward0>)

In [62]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data

tensor([[42.0000,  2.0000, -6.0256,  2.0000],
        [11.8454,  2.0000,  8.4342,  2.0000],
        [ 2.0000, 11.8313,  2.0000, -3.9883],
        [ 2.0000,  2.0000,  9.2210,  2.0000],
        [ 8.4095, -3.7398, 10.7595,  2.0000],
        [ 2.0000,  7.0122,  2.0000, -3.7297],
        [ 2.0000, -3.3394, -5.0609, 10.5741],
        [ 2.0000,  8.7823, -4.4440, -3.2224]])

In [63]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
